In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import yaml

from recbole.quick_start import run_recbole
# from recbole.quick_start import objective_function
from recbole.config import Config
from recbole.data import data_preparation, create_dataset
from recbole.trainer import HyperTuning
from recbole.utils import (
    get_model,
    get_trainer,
    init_seed
)

import utils
from real_temporal import TimeCutoffDataset

# 1. Declarations & Definitions

## 1.1. Define flags and global variables

In [3]:
use_TimeCutoff = True
reproducible = True
seed = 42

## 1.2. Define configurations

Configuration for data, model, training and evaluation

In [4]:
config_dict = {
    # For model
    'model': "NPE",
    'embedding_size': 64,
    'dropout_prob': 0.2,
    'loss_type': "CE",

    # For data
    'dataset': 'ml-100k',
    'load_col': {"inter": ['user_id', 'item_id', 'timestamp']},
    'use_TimeCutoff': use_TimeCutoff,

    # For training
    'epochs': 500,
    'train_batch_size': 4096,
    'learning_rate': 0.030752612145593928,
    'train_neg_sample_args': None,
    
    # For evaluation
    'eval_batch_size': 4096,
    'metrics': ["Recall", "MRR", "NDCG", "Hit", "Precision"],
    'topk': 10,
    'valid_metric': 'MRR@10',

    # Environment
    'checkpoint_dir': utils.get_path_dir_ckpt(),
    'device': 'mps',
    'show_progress': True,
    'reproducibility': reproducible,
    'seed': seed,
}

if use_TimeCutoff is True:
    config_dict = {
        **config_dict,
        'eval_args': {
            "order": "TO",
            "split": {"CO": '886349689'},
            "group_by": 'user_id'
        },
    }
else:
    config_dict = {
        **config_dict,
        'eval_args': {
            "order": "TO",
            "split": { "LS": "valid_and_test" },
            "group_by": None
        },
    }

# 2. Train

## 2.1. Declare necessary components for training

In [5]:
config = Config(config_dict=config_dict)

init_seed(config["seed"], config["reproducibility"])

# Define data related things
if use_TimeCutoff:
    dataset = TimeCutoffDataset(config)
else:
    dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)

# Define model
model_name = config['model']
model = get_model(model_name)(config, train_data._dataset).to(config['device'])

# Define trainer
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

In [6]:
logger = utils.get_logger()

print(" === Config === ")
logger.info(config)

print(" === Dataset === ")
logger.info(dataset)

print(" === Model === ")
logger.info(model)

2024-06-19 22:36:25.861 | INFO     | __main__:<module>:4 - 
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 42
state = INFO
reproducibility = True
data_path = /Users/macos/miniforge3/envs/py/lib/python3.10/site-packages/recbole/config/../dataset_example/ml-100k
checkpoint_dir = logs/Jun19_223623/ckpts
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 500
train_batch_size = 4096
learner = adam
learning_rate = 0.030752612145593928
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'CO': '886349689'}, 'order': 'TO', 'group_by': 'user_id', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG',

 === Config === 
 === Dataset === 
 === Model === 


## 2.2. Start training

In [7]:
best_valid_score, best_valid_result = trainer.fit(
    train_data, 
    valid_data,
    verbose=True,
    show_progress=config["show_progress"]
)

print("** Validation result")
print(f"best_valid_score: {best_valid_score:.4f}")
for metric, val in best_valid_result.items():
    print(f"{metric:<15}: {val:.4f}")

Evaluate   : 100%|███████████████████████████████████████████████████| 1/1 [00:00<00:00, 215.24it/s]


** Validation result
best_valid_score: 0.0211
recall@10      : 0.0634
mrr@10         : 0.0211
ndcg@10        : 0.0309
hit@10         : 0.0634
precision@10   : 0.0063


## 3.3. Start testing

In [8]:
test_result = trainer.evaluate(test_data)

print("** Test result")
for metric, val in test_result.items():
    print(f"{metric:<15}: {val:.4f}")

** Test result
recall@10      : 0.0236
mrr@10         : 0.0043
ndcg@10        : 0.0087
hit@10         : 0.0236
precision@10   : 0.0024


# 4. Tune hyper params

## 4.1. Define hyper params and object function

In [9]:
hyper_params = {
    'loguniform': {
        'learning_rate': [-8, 0]
    },
    'choice': {
        'embedding_size': [64],
        'dropout_prob': [0.2, 0.3, .5]
    }
}

In [10]:
def objective_function(config_dict=None, config_file_list=None):
    config = Config(
        config_dict=config_dict,
        config_file_list=config_file_list,
    )

    init_seed(config["seed"], config["reproducibility"])

    # Define data related things
    if use_TimeCutoff:
        dataset = TimeCutoffDataset(config)
    else:
        dataset = create_dataset(config)
    train_data, valid_data, test_data = data_preparation(config, dataset)

    # Define model
    model_name = config['model']
    model = get_model(model_name)(config, train_data._dataset).to(config['device'])

    # Define trainer
    trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)

    # Start training
    best_valid_score, best_valid_result = trainer.fit(train_data, valid_data, verbose=False)

    # Start evaluating
    test_result = trainer.evaluate(test_data)

    return {
        'model': model_name,
        'best_valid_score': best_valid_score,
        'valid_score_bigger': config['valid_metric_bigger'],
        'best_valid_result': best_valid_result,
        'test_result': test_result
    }

## 4.2. Save config as yaml file

In [11]:
with open(utils.get_path_conf(), 'w+') as f:
    yaml.dump(config_dict, f, allow_unicode=True)

## 4.2. Start tuning

In [12]:
tuning_algo = "bayes"
early_stop = 10
max_evals = 10

hp = HyperTuning(
    objective_function=objective_function,
    algo=tuning_algo,
    early_stop=early_stop,
    max_evals=max_evals,
    params_dict=hyper_params,
    fixed_config_file_list=[utils.get_path_conf()]
)


hp.run()

running parameters:                                   
{'dropout_prob': 0.2, 'embedding_size': 64, 'learning_rate': 0.06678097290014981}
current best valid score: 0.0150                      
current best valid result:                            
OrderedDict([('recall@10', 0.0479), ('mrr@10', 0.015), ('ndcg@10', 0.0224), ('hit@10', 0.0479), ('precision@10', 0.0048)])
current test result:                                  
OrderedDict([('recall@10', 0.0), ('mrr@10', 0.0), ('ndcg@10', 0.0), ('hit@10', 0.0), ('precision@10', 0.0)])
running parameters:                                                 
{'dropout_prob': 0.2, 'embedding_size': 64, 'learning_rate': 0.0021038267305494214}
current best valid score: 0.0196                                    
current best valid result:                                          
OrderedDict([('recall@10', 0.0634), ('mrr@10', 0.0196), ('ndcg@10', 0.0294), ('hit@10', 0.0634), ('precision@10', 0.0063)])
current test result:                               

## 4.3. Export tunning result

In [13]:
# print best parameters
print('best params: ', hp.best_params)

# print best result
print('best result: ')
print(hp.params2result[hp.params2str(hp.best_params)])

# export to JSON file
tune_result = {
    'best_params': hp.best_params,
    'best_result': hp.params2result[hp.params2str(hp.best_params)]
}
with open(utils.get_path_tune_log(), "w+") as f:
    json.dump(tune_result, f, indent=2, ensure_ascii=False)

best params:  {'dropout_prob': 0.2, 'embedding_size': 64, 'learning_rate': 0.030752612145593928}
best result: 
{'model': 'NPE', 'best_valid_score': 0.0211, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('recall@10', 0.0634), ('mrr@10', 0.0211), ('ndcg@10', 0.0309), ('hit@10', 0.0634), ('precision@10', 0.0063)]), 'test_result': OrderedDict([('recall@10', 0.0236), ('mrr@10', 0.0043), ('ndcg@10', 0.0087), ('hit@10', 0.0236), ('precision@10', 0.0024)])}


# 5. Offline evaluation

In [ ]:
EVAL_MODES = {
    "temporal": "TO",
    "leave1out": "LS"
}

In [ ]:
selected_eval_models = [EVAL_MODES[k] for k in ["temporal", "leave1out"]]
selected_eval_metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
selected_topk = 10

eval_args = {
    "order": "TO",
    "split": {
        'LS': "valid_and_test"
    },
    "group_by": None
}

config_dict = {
    'eval_args': eval_args,
    # 'train_neg_sample_args': None,
}

run_recbole(model=model, dataset=data, config_dict=config_dict)

In [ ]:
run_recbole(model='NPE', dataset='ml-100k', config_file_list=['test.yml'])